In [3]:
import pandas as pd
import statsmodels.api as sapi
import numpy as np
import statsmodels.tsa.api as tsaapi

In [4]:
import os

In [5]:
class CointModel(object):
    def __init__(self,X1,X2):
        '''
        一般来说将X1作为因变量，将X2作为自变量
        尽量提供对数价格
        '''
        self.X1 = X1
        self.X2 = X2
        self.b = 0 #截距项
        self.beita = 1 #系数项
        self.e = [] #残差
        
    def adf_test_x1_x2(self):
        '''
        检验X1，X2是否为同阶单整序列
        返回 X1,X2 adf 检验的P值
        '''
        result_x1 = tsaapi.stattools.adfuller(self.X1)
        result_x2 = tsaapi.stattools.adfuller(self.X2)
        return result_x1[1],result_x2[1]
        
    def adf_test_dx1_dx2(self):
        '''
        检验差分后的X1，X2是否是平稳的
        返回 dX1，dX2 adf 检验的P值
        '''
        result_dx1 = tsaapi.stattools.adfuller(pd.Series(self.X1).diff().dropna())
        result_dx2 = tsaapi.stattools.adfuller(pd.Series(self.X2).diff().dropna())
        return result_dx1[1],result_dx2[1]
    
    def fit(self,print_summary=False):
        '''
        print_summary:bool，是否输出回归结果
        返回 b,beita，残差
        '''
        ###拟合，
        self.X2 = sapi.add_constant(self.X2)
        ols_model = sapi.OLS(self.X1,self.X2)
        ols_result = ols_model.fit()
        if print_summary:
            print(ols_result.summary2())
        self.b = ols_result.params[0]
        self.beita = ols_result.params[1]
        self.e = ols_result.resid
        return self.b,self.beita,self.e
    
    def resid_stationary_test(self,maxlag=None):
        '''
        检验残差是否平稳
        maxlag:确定最大滞后项，若无则自动设置AIC为准则
        返回 adf检验统计量，Pvalue，滞后阶数，critical values
        '''
        adfresult = tsaapi.stattools.adfuller(self.e,maxlag=maxlag)
        return adfresult[0],adfresult[1],adfresult[2],adfresult[4]

In [8]:
files = os.listdir("./data_without_formula/")
files_path = [('./data_without_formula/'+f) for f in files]
company_name = [f[:-5] for f in files]
#####  注意这里files_path与company_name 的内容是一一对应的

In [11]:
coint_result = pd.DataFrame(index=company_name,columns=['HK_p_value','SH_p_value','D_HK_p_value','D_SH_p_value','b','beita','sample_e_p_value','total_sample_e_p_value'])
##批量协整检验
for index,file_path in enumerate(files_path):
    data = pd.read_excel(file_path,index_col=0)
    H_code = data.columns[0]
    A_code = data.columns[1]
    data.drop(columns=['前收盘价','涨停价','跌停价'],inplace=True)
    data = data[~data['交易状态'].isin(['停牌一天'])]
    '''
    样本内
    '''
    sample_data = data.loc[:pd.datetime(2019,4,16)]
    ###估计协整模型得到系数及残差
    sample_H = np.log(sample_data.iloc[:,0])
    sample_A = np.log(sample_data.iloc[:,1])
    coint_model = CointModel(sample_H,sample_A)
    b,beita,sample_e = coint_model.fit()
    '''
    样本外
    '''
    out_sample_data = data.loc[pd.datetime(2019,4,17):]
    out_sample_H = np.log(out_sample_data.iloc[:,0])
    out_sample_A = np.log(out_sample_data.iloc[:,1])
    out_sample_e = out_sample_H - b - beita*out_sample_A
    
    
    ###创建协整模型,X1为港股，X2为A股
    coint_model = CointModel(sample_H,sample_A)
    ###估计模型，并将结果填入coint_result
    company = company_name[index]
    print(company)
    coint_result["HK_p_value"][company],coint_result["SH_p_value"][company] = coint_model.adf_test_x1_x2()
    
    coint_result["D_HK_p_value"][company],coint_result["D_SH_p_value"][company] = coint_model.adf_test_dx1_dx2()
    
    coint_result["b"][company],coint_result["beita"][company] = coint_model.fit()[0],coint_model.fit()[1]
    
    coint_result["sample_e_p_value"][company] = coint_model.resid_stationary_test()[1]
    
    '''
    使用样本内参数估计样本外模型后使用全样本的残差进行单位根检验
    '''
    coint_result["total_sample_e_p_value"][company] = tsaapi.stattools.adfuller(sample_e.append(out_sample_e))[1]
coint_result.to_excel("所有配对协整检验结果新增全样本残差单位根检验.xlsx")

一拖股份
万科A
上海医药
上海电气
上海石化
东方电气
东方航空
东方证券
中信证券
中信银行
中兴通讯
中国中冶
中国中车
中国中铁
中国交建
中国人寿
中国国航
中国太保
中国平安
中国石化
中国石油
中国神华
中国铁建
中国铝业
中国银河
中国银行
中海油服
中煤能源
中联重科
中远海发
中远海控
中远海能
中集集团
丽珠集团
交通银行
光大证券
光大银行
兖州煤业
农业银行
创业环保
北辰实业
华泰证券
华电国际
华能国际
南京熊猫
南方航空
四川成渝
国泰君安
复星医药
大众公用
大唐发电
大连港
宁沪高速
工商银行
广发证券
广汽集团
广深铁路
建设银行
招商证券
招商银行
新华保险
晨鸣纸业
比亚迪
民生银行
江西铜业
洛阳钼业
海信家电
海螺水泥
海通证券
深高速
潍柴动力
白云山
皖通高速
福耀玻璃
紫金矿业
郑煤机
金隅集团
金风科技
长城汽车
青岛啤酒
鞍钢股份
马钢股份


In [3]:
####中国国航协整检验
def coint_test_zhongguoguohang():
    data = pd.read_excel('./data_without_formula/马钢股份.xlsx',index_col=0)
    H = np.log((data.loc[:pd.datetime(2019,4,16)]).iloc[:,0])
    A = np.log((data.loc[:pd.datetime(2019,4,16)]).iloc[:,1])
    DH = np.log(H).diff().dropna()
    DA = np.log(A).diff().dropna()
    print('中国国航A股单位根检验',tsaapi.stattools.adfuller(A))
    print('中国国航港股单位根检验',tsaapi.stattools.adfuller(H))
    print('中国国航A股差分单位根检验',tsaapi.stattools.adfuller(DA))
    print('中国国航港股差分单位根检验',tsaapi.stattools.adfuller(DH))
    ##OLS回归
    model = sapi.OLS(H,sapi.add_constant(A))
    result = model.fit()
    print(result.summary2())
    resid = result.resid
    print(tsaapi.stattools.adfuller(resid))
    pd.Series(resid).to_excel("E.xlsx")
coint_test_zhongguoguohang()

中国国航A股单位根检验 (-2.664028917964746, 0.08047233614673077, 0, 303, {'1%': -3.4521175397304784, '5%': -2.8711265007266666, '10%': -2.571877823851692}, -1377.2338820953894)
中国国航港股单位根检验 (-2.9187388572129587, 0.043211915038267816, 0, 303, {'1%': -3.4521175397304784, '5%': -2.8711265007266666, '10%': -2.571877823851692}, -1283.8867061766734)
中国国航A股差分单位根检验 (-18.80696061570859, 2.0225900784579675e-30, 0, 302, {'1%': -3.4521902441030963, '5%': -2.871158406898617, '10%': -2.5718948388228586}, -1480.6311247645522)
中国国航港股差分单位根检验 (-17.98737986263998, 2.761735970195612e-30, 0, 302, {'1%': -3.4521902441030963, '5%': -2.871158406898617, '10%': -2.5718948388228586}, -1384.1464267020692)
                  Results: Ordinary least squares
Model:              OLS              Adj. R-squared:     0.717     
Dependent Variable: 0323.HK          AIC:                -1059.6067
Date:               2020-04-24 19:52 BIC:                -1052.1727
No. Observations:   304              Log-Likelihood:     531.80    
Df 

D:\Anaconda\lib\site-packages\numpy\core\fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [6]:
####中国国航协整检验
def coint_test_zhongguoguohang():
    data = pd.read_excel('./data_without_formula/东方航空.xlsx',index_col=0)
    H = np.log(data.iloc[:,0])
    A = np.log(data.iloc[:,1])
    DH = np.log(H).diff().dropna()
    DA = np.log(A).diff().dropna()
    print('中国国航A股单位根检验',tsaapi.stattools.adfuller(A))
    print('中国国航港股单位根检验',tsaapi.stattools.adfuller(H))
    print('中国国航A股差分单位根检验',tsaapi.stattools.adfuller(DA))
    print('中国国航港股差分单位根检验',tsaapi.stattools.adfuller(DH))
    ##OLS回归
    model = sapi.OLS(H,sapi.add_constant(A))
    result = model.fit()
    print(result.summary2())
    resid = result.resid
    print(tsaapi.stattools.adfuller(resid))
coint_test_zhongguoguohang()

中国国航A股单位根检验 (-1.6335460929239154, 0.4656350973982443, 10, 529, {'1%': -3.442772146350605, '5%': -2.8670191055991836, '10%': -2.5696881663873414}, -2351.6139966672586)
中国国航港股单位根检验 (-1.1114429587754016, 0.710487078931478, 0, 539, {'1%': -3.4425405682241816, '5%': -2.8669171671779816, '10%': -2.5696338432333636}, -2115.8019299726016)
中国国航A股差分单位根检验 (-8.392247336686944, 2.3495579642889983e-13, 9, 529, {'1%': -3.442772146350605, '5%': -2.8670191055991836, '10%': -2.5696881663873414}, -2924.4536590503167)
中国国航港股差分单位根检验 (-23.823478363343565, 0.0, 0, 538, {'1%': -3.442563336759378, '5%': -2.866927190004947, '10%': -2.5696391843672695}, -2546.6372850227303)
                  Results: Ordinary least squares
Model:              OLS              Adj. R-squared:     0.812     
Dependent Variable: 0670.HK          AIC:                -1061.4583
Date:               2020-04-25 18:29 BIC:                -1052.8752
No. Observations:   540              Log-Likelihood:     532.73    
Df Model:           1 

D:\Anaconda\lib\site-packages\numpy\core\fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
